In [1]:




import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger, WandbTestLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger

device = 'cuda'

# Create dataset
dm = DataModule(
    name=["ribo-HQ"],
    data_type=["dms", "shape"],
    force_download=False,
    batch_size=128,
    num_workers=0,
    train_split=512,
    valid_split=0,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
)

dm.setup('fit')


ribo-HQ: Loading data.pkl...


In [ ]:

model = create_model(
    model="evoformer",
    data="multi",
    quality=True,
    ntoken=5,
    d_model=64,
    c_z=8,
    num_blocks=4,
    no_recycles=0,
    dropout=0,
    lr=3e-3,
    weight_decay=0,
    gamma=0.997,
).to(device)


trainer = Trainer(
    max_epochs=200,
    callbacks=[
        WandbFitLogger(dm=dm, batch_size=128, log_every_n_epoch=1),
        WandbTestLogger(dm=dm),
        # LearningRateMonitor(logging_interval="step"),
    ],
    logger=WandbLogger(project="dmsensei", log_model=True),
)


In [ ]:
import torch, wandb
import numpy as np
from torch.nn.functional import mse_loss
# wandb.init(project="dmsensei", name="debug-transformer", tags=["debug", "transformer"])
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
for epoch in range(1000):
    losses = []
    maes = []
    for batch in dm.train_dataloader():
        pred = model(batch.get('sequence'))
        dms = pred['dms'][batch.get_index('dms')], batch.get('dms')
        shape = pred['shape'][batch.get_index('shape')], batch.get('shape')
        true = torch.cat([dms[1], shape[1]], dim=0)
        pred = torch.cat([dms[0], shape[0]], dim=0)
        mask = true!=-1000.
        loss = mse_loss(pred[mask], true[mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        maes.append((pred[mask]-true[mask]).abs().mean().item())
    print(f"Epoch {epoch} loss: {np.sqrt(np.mean(losses)).round(3)} mae: {np.mean(maes).round(3)} \r", end="")
    # wandb.log({"loss": np.sqrt(np.mean(losses)).round(3), "epoch": epoch, "mae": np.mean(maes).round(3)})
        